In [1]:
import numpy as np # Linear Algebra
import matplotlib.pyplot as plt # Plotting

In [ ]:
# Full Kalman filter N-D
def kalman_step(mu, sig, u, z, A = np.eye(2), B = np.eye(2), C = np.eye(2), R = np.eye(2) * 0.1, Q = np.eye(2) * 0.1):
    mu_bar = A @ mu + B @ u
    sig_bar = A @ sig @ A.T + R
    K = sig_bar @ C.T @ np.linalg.inv(C @ sig_bar @ C.T + Q)
    mu_new = mu_bar + K @ (z - C @ mu_bar)
    sig_new = (np.eye(*sig.shape) - K @ C) @ sig_bar
    return mu_new, sig_new

In [ ]:
# Let's instantiate a 2D double integrator system!
# Equations of Motion!
dt = 0.05

# Control affine/linear: x' = A @ x + B @ u
A = np.array([[1., 0., dt, 0.], [0., 1., 0., dt], [0., 0., 1., 0.], [0., 0., 0., 1.]])
B = np.array([[dt*dt, 0.], [0., dt*dt], [dt, 0.], [0., dt]])
R = np.eye(4) * 0.001 # noise for movement! (1mm variance!)

# Observation model - we can only "see" the position, not the velocity
C = np.array([[1., 0., 0., 0.], [0., 1., 0., 0.]])
Q = np.eye(2) * 0.1 # noise for observation! (10cm variance!)

In [ ]:
# Let's do a simulation!
x = np.zeros((4, 1)) # We start at (0,0) with zero velocity
u = np.array([[0.05], [-0.1]])

K = 200 # K steps

states = [np.copy(x)]
estimation = [(np.copy(x), np.eye(4) * 0.001)] # We start with a small uncertainty!

for k in range(K):
    # u = np.array([[0.05 * np.sin(2.*k*dt)], [-0.1 * np.sin(2.*k*dt)]])
    # Let's move the simulator forward
    x = A @ x + B @ u
    states.append(np.copy(x))
    # Let's get the measurement!
    z = C @ x + np.random.multivariate_normal(np.zeros((2,)), Q) # with noise!
    # Let's do a Kalman step!
    mu, sig = estimation[-1]
    mu_new, sig_new = kalman_step(mu, sig, u, z, A, B, C, R, Q)
    estimation.append((np.copy(mu_new), np.copy(sig_new)))

In [ ]:
# Let's plot!
times = np.arange(len(states))

# Positions
plt.plot(times, [st[0] for st in states], label='X-Actual')
plt.plot(times, [st[0][0, 0] for st in estimation], label='X-Estimation')

plt.plot(times, [st[1] for st in states], label='Y-Actual')
plt.plot(times, [st[0][1, 0] for st in estimation], label='Y-Estimation')

# # Velocities
# plt.plot(times, [st[2] for st in states], label='X-Actual')
# plt.plot(times, [st[0][2, 0] for st in estimation], label='X-Estimation')

# plt.plot(times, [st[3] for st in states], label='Y-Actual')
# plt.plot(times, [st[0][3, 0] for st in estimation], label='Y-Estimation')

plt.legend()

plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

# Our 2-dimensional distribution will be over variables X and Y
N = 40
X = np.linspace(-6, 6, N)
Y = np.linspace(-6, 6, N)
X, Y = np.meshgrid(X, Y)

# Mean vector and covariance matrix
idx = -1
mu = estimation[idx][0][:2, 0]
Sigma = estimation[idx][1][:2, :2]

# Pack X and Y into a single 3-dimensional array
pos = np.empty(X.shape + (2,))
pos[:, :, 0] = X
pos[:, :, 1] = Y

def multivariate_gaussian(pos, mu, Sigma):
    """Return the multivariate Gaussian distribution on array pos."""

    n = mu.shape[0]
    Sigma_det = np.linalg.det(Sigma)
    Sigma_inv = np.linalg.inv(Sigma)
    N = np.sqrt((2*np.pi)**n * Sigma_det)
    # This einsum call calculates (x-mu)T.Sigma-1.(x-mu) in a vectorized
    # way across all the input variables.
    fac = np.einsum('...k,kl,...l->...', pos-mu, Sigma_inv, pos-mu)

    return np.exp(-fac / 2) / N

# The distribution on the variables X, Y packed into pos.
Z = multivariate_gaussian(pos, mu, Sigma)

# plot using subplots
fig = plt.figure()
ax1 = fig.add_subplot(2,1,1,projection='3d')

ax1.plot_surface(X, Y, Z, rstride=3, cstride=3, linewidth=1, antialiased=True,
                cmap=cm.viridis)
ax1.view_init(55,-70)
ax1.set_xticks([])
ax1.set_yticks([])
ax1.set_zticks([])
ax1.set_xlabel(r'$x_1$')
ax1.set_ylabel(r'$x_2$')

ax2 = fig.add_subplot(2,1,2,projection='3d')
ax2.contourf(X, Y, Z, zdir='z', offset=0, cmap=cm.viridis)
ax2.view_init(90, 270)

ax2.grid(False)
ax2.set_xticks([])
ax2.set_yticks([])
ax2.set_zticks([])
ax2.set_xlabel(r'$x_1$')
ax2.set_ylabel(r'$x_2$')

plt.show()